0. Import Library

In [ ]:
import math
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler, ConcatDataset, Dataset
from torch.utils.data.dataset import random_split
from torcheval.metrics.functional import binary_auroc
from sklearn.metrics import roc_auc_score

from utils.rnn_attention_kd import *

1. Checking the Device

In [ ]:
# Checking whether the device used for training is CPU or GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

2. Training the Teacher Model

In [ ]:
# Setting the hyperparametes of teacher model.
input_size = 12 # Number of feature on this study
hidden_size = 4 # The hidden units on each layer on RNN
num_layers = 1 # The number of layers of RNN
num_classes = 2 # The number of class (At-risk and No-risk students)

In [ ]:
# Setting a model
model_t = TeacherGRU(input_size=input_size, 
                     hidden_size=hidden_size, 
                     num_layers=num_layers, 
                     num_classes=num_classes)

model_t = model_t.to(device)

In [ ]:
# Training the teacher model
train_teacher(model=model_t, 
              train_loader=train_loader,
              test_loader=test_loader, 
              epochs=150, 
              device=device, 
              train_graph_path=None, 
              learning_rate=0.01, 
              weight_decay=1e-5, 
              scheduler_funnction=False, 
              print_results=True)

In [ ]:
# Evaluation the performance of teacher model with test set
precision, recall, f1, accuracy, auc, predict_value = model_evaluation_teacher(model=model_t, 
                                                                               test_loader=test_loader, 
                                                                               device=device, threshold=0.5,
                                                                               confusion_matrix=True, 
                                                                               print_results=False)

In [ ]:
# Save the teacher weight model
PATH_T = f'save/{file_path_teacher}/teacher_model'
torch.save(model_t.state_dict(), PATH_T)